TOPIC RECOGNITION

ci sono 3 importanti parametri:
-numero di topic
-numero di parole per topic
-numero di topic per documento

STEP:
-TOKENIZZARE E RIPULIRE OGNI DOCUMENTO (QUINDI OGNI POST)(utilizziamo tokenizer per fare prima questa volta)
Lasciare #parola?? (boh decidere)

-per farlo con LSA il prof usa sklearn



In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

In [1]:
import csv
from collections import Counter
from tkinter.tix import COLUMN
import matplotlib.pyplot as plt
import re
import pandas as pd
from math import log2
import spacy
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import gensim, pprint



[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
corpus=["https://colab.research.google.com/drive/1dduOM7M3RubSeqpogrM1H7PhL2S9ZMbt#scrollTo=1eeVrjeBpn1x",
        "is not happy capitol riot vote forces , not happy Republican to pick truth or Trump not good is not yours",
        "EU not good agrees to allow entry to vaccinated travelers",
        "Why the United Nation is stuck on the Israeli-Palestinian conflict",
        "New York attorney general adds 'criminal capacity' to probe of Trump Organization",
        "Given the recent accusations against the European Commission on land registry, beach resorts, PNRR, public accounts, etc., it is clear that many think that the EU should behave as if it were our ATM that gives money and is silent. But the ATM gives you your money, the EU gives us other people's money.",
        "ANSA FORUM | Lega secretary Matteo Salvini responds live to questions from director Luigi Contu and colleague of the political-parliamentary editorial staff Michela Suglia #ANSA",
        "Murder Willy, Bianchi in the courtroom: I didn't even touch him with a finger",
        "The positives at #vaiolodellescimmie in Lombardy have risen to three. The Region has already communicated the protocol that hospitals must follow in the event of symptoms appearing",
        "An assault rifle in the hand of an 18-year-old. An assault rifle. #Texas",
        "Transfer market live: Pogba-Juve in the final close, Napoli and Fiorentina on Deulofeu [by Giulio Cardone, Enrico De Lellis]",
        "#Economia circular and #gestione environmental: limits and opportunities. The #aziende must carefully screen #investimenti , #costi and perspectives and must show #flessibilità and a rapid propensity for change",
        "Fire on the island of Stromboli, flames extinguished after a night of fire #ANSA",
        "A new anti-cancer vaccine, tested in mice and monkeys, has been shown to be able to break down the defenses that tumors put in place to protect themselves from attacks by the immune system by blocking diseased cells",
        "Term of the day: Computational fluid dynamics Definition: The numerical solution of flow equations in practical problems such as aircraft design or hydraulic structures.",
        "The mayor of Kiev: 'The Russian war will not stop in Ukraine'",
        "The killer of elementary school in #Uvalde in #Texas Salvador Ramos worked in a restaurant not far from the scene of the massacre. He would have been able to save around 4,000 dollars (3.7 thousand euros) to buy the weapons he used in the massacre"]

In [3]:
my_not=set()
for frase in corpus[1:]:
  regex=re.compile("not \w+")
  for a in regex.findall(frase):
    my_not.add(a)


In [ ]:
my_not

{'not far', 'not good', 'not happy', 'not stop', 'not yours'}

In [4]:
pattern_link="https\S+"
pattern_menzione="@\S+"
#list_of_word=['impressed','sure','good','bad','happy','worry','able','fast','super','know','peaceful','cool','sad','confused','cool','attractive','gracious','sick','best','fan','care','love','sunny','funny','dead','annoying','easy','great','cold','regret','sick','like','hard','fun','enjoying','nice','bored','true','mad','angry','afraid']

tmp=''
lemmatized_string=[]


sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words
wnl = WordNetLemmatizer()

In [ ]:
len(all_stopwords)

326

In [5]:
def remove(doc):
  #parole_esaminate=list()
  doc=re.sub(pattern_link, '',doc)
  doc=re.sub(pattern_menzione," ",doc)
  
  return doc


unisco il not con good o altri aggettivi perchè bigram mi unisce le parole 'not' con 'good' se almeno ce ne sono come minimo due e inoltre non posso levare le stop word perchè mi leva anche not prima di fare bigram, anche perchè se non lo faccio poi mi unisce 'is not' se ce ne sono almeno due

In [6]:
tokens_=[]
#parole_esaminate=list()
lemmatized_string=list()
post=list()

tokens_ = [list(gensim.utils.tokenize(remove(doc), lower=True)) for doc in corpus]

for lista in tokens_:
  #if len(lista)>0: #per togliermi quelle vuote che magari contenevano solo link
  parole_esaminate=[]
  
  for parole in lista:
    #if not parole in all_stopwords:
          if parole == 'not':
            tmp='not_'
          elif tmp=='not_' and parole not in all_stopwords:
              parole=tmp+parole
              #parole_esaminate.append(parole)
              tmp=''
          if parole != 'not' and parole not in all_stopwords:
            parole_esaminate.append(parole)

  lemmatized_string.append([wnl.lemmatize(words) for words in parole_esaminate ]) 

bigram_mdl = gensim.models.phrases.Phrases(lemmatized_string, min_count=1, threshold=2)##farglielo fare su tutto il corpus tokenizzato altrimenti non funziona se lo fai su una sola frase
for l in lemmatized_string:

  bigrams = bigram_mdl[l]
  post.append(bigrams)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
post

VETTORIZZARE I DOCUMENTI CON TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [8]:

lemmatized_bigram_string=[' '.join(word) for word in post]

In [ ]:
lemmatized_bigram_string

['',
 'not_happy capitol riot vote force not_happy republican pick truth trump not_good',
 'not_eu not_good agrees allow entry vaccinated traveler',
 'united nation stuck israeli palestinian conflict',
 'new york attorney general add criminal capacity probe trump organization',
 'given recent accusation european commission land registry beach resort pnrr public account etc clear think eu behave atm_give money silent atm_give money eu give people s money',
 'ansa forum lega secretary matteo salvini responds live question director luigi contu colleague political parliamentary editorial staff michela suglia ansa',
 'murder willy bianchi courtroom didn t touch finger',
 'positive vaiolodellescimmie lombardy risen region communicated protocol hospital follow event symptom appearing',
 'assault_rifle hand year old assault_rifle texas',
 'transfer market live pogba juve final close napoli fiorentina deulofeu giulio cardone enrico de lellis',
 'economia circular gestione environmental limit op

In [9]:
modello= TfidfVectorizer()
M=modello.fit_transform(lemmatized_bigram_string)

In [ ]:
M.shape

(17, 188)

In [ ]:
M.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.21436118, 0.        ,
        0.        ]])

In [ ]:
modello.get_feature_names()

In [ ]:
len(modello.get_feature_names())

n_components è il nostro k
l'algoritmo è iterativo qui
c'è una libreria ancora più adeguata per i testi

In [10]:
from sklearn.decomposition import TruncatedSVD

lsa = TruncatedSVD(n_components=10,n_iter=300)
lsa.fit(M)

TruncatedSVD(n_components=10, n_iter=300)

In [ ]:
lsa.components_

array([[-7.37843588e-14,  6.72205189e-18,  6.39541270e-18, ...,
        -1.27228912e-13,  6.81740376e-17,  6.81740376e-17],
       [ 1.16396828e-01,  2.42960739e-15,  2.39725983e-15, ...,
         2.08072822e-01, -7.26735775e-19, -7.26735775e-19],
       [-4.17455883e-15,  1.02904045e-01,  1.02904045e-01, ...,
        -4.29482847e-15, -6.47432027e-21, -6.47432027e-21],
       ...,
       [-3.72626624e-02,  7.93044625e-16,  7.17401528e-16, ...,
         1.76740175e-16,  1.39562132e-17,  1.39562132e-17],
       [-5.18798609e-02,  1.46032231e-16,  1.28762764e-16, ...,
        -1.39732684e-16, -3.82114173e-17, -3.82114173e-17],
       [ 3.99308219e-03, -2.34777650e-16, -1.96275406e-16, ...,
        -1.36584972e-16,  1.16399319e-17,  1.16399319e-17]])

10 sono le direzioni di troncamento
200 è il nostro insieme di parole

quindi ho una codifica delle nostrre parole nei 10 assi

In [ ]:
lsa.components_.shape

(10, 188)

In [11]:
termini=modello.get_feature_names()
print(termini)

['able', 'account', 'accusation', 'add', 'agrees', 'aircraft', 'allow', 'ansa', 'anti', 'appearing', 'assault_rifle', 'atm_give', 'attack', 'attorney', 'aziende', 'beach', 'behave', 'bianchi', 'blocking', 'break', 'buy', 'cancer', 'capacity', 'capitol', 'cardone', 'carefully', 'cell', 'change', 'circular', 'clear', 'close', 'colleague', 'commission', 'communicated', 'computational', 'conflict', 'contu', 'costi', 'courtroom', 'criminal', 'day', 'de', 'defense', 'definition', 'design', 'deulofeu', 'didn', 'director', 'diseased', 'dollar', 'dynamic', 'economia', 'editorial', 'elementary', 'enrico', 'entry', 'environmental', 'equation', 'etc', 'eu', 'euro', 'european', 'event', 'extinguished', 'final', 'finger', 'fiorentina', 'fire', 'flame', 'flessibilità', 'flow', 'fluid', 'follow', 'force', 'forum', 'general', 'gestione', 'giulio', 'give', 'given', 'hand', 'hospital', 'hydraulic', 'immune', 'investimenti', 'island', 'israeli', 'juve', 'kiev', 'killer', 'land', 'lega', 'lellis', 'limit',

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ora voglio associare le parole alle nostre componenti.
quindi abbiamo il nostro elemento e il vettore corrispondente

In [ ]:
for i,j in enumerate(lsa.components_):
  print(i,j)

ora prendo la parola e il valore del concetto e li associo

con lsa parole simili dovrebbero avere lo stesso vettore

siccome 200 sono un pò troppe mi prendo ad esempio le prime 10 parole più rilevanti per il mio asse concettuale

l'informazione con il segno negativo è comunque rilevante quindi potrei prendere il modulo dei valori

In [12]:
for i,j in enumerate(lsa.components_):
  parolaChiave=zip(termini,abs(j))
paroleOrdinate = sorted(parolaChiave,key=lambda x:x[1], reverse = True)
paroleOrdinate = paroleOrdinate[:10]

In [45]:
paroleOrdinate

('russian', 0.19532322340331493)

In [ ]:
for i in paroleOrdinate:
  print(i)

('computational', 0.14287182875138013)
('day', 0.14287182875138013)
('definition', 0.14287182875138013)
('design', 0.14287182875138013)
('dynamic', 0.14287182875138013)
('equation', 0.14287182875138013)
('flow', 0.14287182875138013)
('fluid', 0.14287182875138013)
('hydraulic', 0.14287182875138013)
('numerical', 0.14287182875138013)


In [ ]:
len(paroleOrdinate)

10

siccome l'asse non è più una parola ma è indotto dall'lsa sto tentando di etichettare ciscun asse secondo il valore..poichè le colonnne di U sono la nostra base se considero le parole, quindi la riga iesima è associata ad una parola, quindi prendo i valori più alti delle componenti di un singolo asse e catturo la porzione di parola che è associata di più per l'asse.
la parola può essere ripetuta negli altri assi proprio perchè ciascun asse non è più associato ad una singola parola

In [ ]:

for i,j in enumerate(lsa.components_):
  parolaChiave=zip(termini,abs(j))
  paroleOrdinate = sorted(parolaChiave,key=lambda x:x[1], reverse = True)
  paroleOrdinate = paroleOrdinate[:10]
  print("Asse concettuale n. ",i, ": ")
  for k in paroleOrdinate:
    print(k)
  print('\n\n')


ci creiamo un dizionario con le parole asociate agli assi

In [ ]:
concetti={}
#usare questo quelli di prima sono prove
for i,j in enumerate(lsa.components_):
  parolaChiave=zip(termini,abs(j))
  paroleOrdinate = sorted(parolaChiave,key=lambda x:x[1], reverse = True)
  paroleOrdinate = paroleOrdinate[:10]
  concetti["concetto_" + str(i)]=[]
  print("Asse concettuale n. ",i, ": ")
  for k in paroleOrdinate:
    concetti["concetto_" + str(i)].append(k)
    print(k)
  print('\n\n')

In [ ]:
concetti

{'concetto_0': [('not_happy', 0.34253734485146925),
  ('not_good', 0.2945522174674141),
  ('trump', 0.2902105029711441),
  ('new', 0.19544515300786056),
  ('capitol', 0.17126867242573462),
  ('force', 0.17126867242573462),
  ('pick', 0.17126867242573462),
  ('republican', 0.17126867242573462),
  ('riot', 0.17126867242573462),
  ('truth', 0.17126867242573462)],
 'concetto_1': [('ansa', 0.4296107681688967),
  ('fire', 0.39699055206882033),
  ('extinguished', 0.19849527603441017),
  ('flame', 0.19849527603441017),
  ('island', 0.19849527603441017),
  ('night', 0.19849527603441017),
  ('stromboli', 0.19849527603441017),
  ('live', 0.18545345258172588),
  ('colleague', 0.1467553077626638),
  ('contu', 0.1467553077626638)],
 'concetto_2': [('assault_rifle', 0.3695762996161747),
  ('texas', 0.27897141345003224),
  ('massacre', 0.2694001374930685),
  ('able', 0.19400753191751557),
  ('hand', 0.18478814980808736),
  ('old', 0.18478814980808736),
  ('year', 0.18478814980808736),
  ('not_good', 0

ora vogliamo prendere le frasi che sono nel nostro corpus e vedere in quali assi concettuali sono meglio mappate

In [ ]:
corpus[14]

'Term of the day: Computational fluid dynamics Definition: The numerical solution of flow equations in practical problems such as aircraft design or hydraulic structures.'

In [15]:
parole=list(gensim.utils.tokenize('the war is a universal conflict ukraine fire', lower=True))#usare poi la funzione che pulisce bene come per iol topic

quindi vediamo come poter individuare i concetti di una certa frase nel nostro spazio ridotto lsa

quindi vado a vedermi le chiavi e mi calcolo il punteggio per ciascuna parola

In [16]:
punteggi=dict()

for key,lista in concetti.items():
  punteggio=0
  for i in parole:
    for concetto_parola,valore in lista:
      if(i == concetto_parola):
        punteggio+=valore
  punteggi[key]=punteggio

In [ ]:
punteggi

{'concetto_0': 0,
 'concetto_1': 0,
 'concetto_2': 0,
 'concetto_3': 0,
 'concetto_4': 0,
 'concetto_5': 0,
 'concetto_6': 0,
 'concetto_7': 0,
 'concetto_8': 0,
 'concetto_9': 1.142974630011041}

In [17]:
def result_Topic(corpus):
  risultato = []
  print("Risultato di ", corpus)
  for concetto,valore in punteggi.items():
    if valore!=0:
      print(concetto, concetti[concetto])
  

In [18]:
result_Topic(corpus[15])

Risultato di  The mayor of Kiev: 'The Russian war will not stop in Ukraine'
concetto_1 [('ansa', 0.42961076816889676), ('fire', 0.3969905520688208), ('extinguished', 0.1984952760344104), ('flame', 0.1984952760344104), ('island', 0.1984952760344104), ('night', 0.1984952760344104), ('stromboli', 0.1984952760344104), ('live', 0.18545345258172544), ('colleague', 0.14675530776266377), ('contu', 0.14675530776266377)]
concetto_5 [('conflict', 0.1927833484451045), ('israeli', 0.1927833484451045), ('nation', 0.1927833484451045), ('palestinian', 0.1927833484451045), ('stuck', 0.1927833484451045), ('united', 0.1927833484451045), ('money', 0.17938473843344024), ('cardone', 0.12713695161832528), ('close', 0.12713695161832528), ('de', 0.12713695161832528)]
concetto_7 [('conflict', 0.17192691085304115), ('israeli', 0.17192691085304115), ('nation', 0.17192691085304115), ('palestinian', 0.17192691085304115), ('stuck', 0.17192691085304115), ('united', 0.17192691085304115), ('war', 0.15823065605135875), 

In [19]:
def result_Topic_max(corpus):
  risultato = []
  massimo = max(punteggi.values())
  print("Risultato di ", corpus)
  for concetto,valore in punteggi.items():
    if valore == massimo:
      risultato.append((concetto, concetti[concetto]))
  return (risultato)

In [20]:
result_Topic_max(corpus[15])

Risultato di  The mayor of Kiev: 'The Russian war will not stop in Ukraine'


[('concetto_8',
  [('kiev', 0.2088064321713682),
   ('mayor', 0.2088064321713682),
   ('not_stop', 0.2088064321713682),
   ('russian', 0.2088064321713682),
   ('ukraine', 0.2088064321713682),
   ('war', 0.2088064321713682),
   ('willy', 0.19908508534554092),
   ('bianchi', 0.19908508534554087),
   ('courtroom', 0.19908508534554087),
   ('didn', 0.19908508534554087)])]

ORA FACCIO LA STESSA COSA APPLICANDO PERO' LDA

In [21]:
from gensim.models import LdaModel, HdpModel

In [22]:
dictionary = gensim.corpora.Dictionary(post)
corpus2 = [dictionary.doc2bow(text) for text in post]

In [ ]:
dictionary.token2id

In [24]:
tfidf = gensim.models.TfidfModel(corpus2)
corpus_tfidf = tfidf[corpus2]

In [ ]:
list(corpus_tfidf)

più si esegue più è preciso

In [27]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus2, num_topics=10, id2word=dictionary, passes=500)

In [28]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.050*"fire" + 0.026*"courtroom" + 0.026*"didn" + 0.026*"traveler" + 0.026*"bianchi"')
(1, '0.038*"new" + 0.038*"add" + 0.038*"york" + 0.038*"criminal" + 0.038*"attorney"')
(2, '0.033*"money" + 0.022*"eu" + 0.022*"atm_give" + 0.012*"dynamic" + 0.012*"equation"')
(3, '0.027*"kiev" + 0.027*"ukraine" + 0.027*"war" + 0.027*"not_stop" + 0.027*"mayor"')
(4, '0.070*"not_happy" + 0.036*"not_good" + 0.036*"trump" + 0.036*"riot" + 0.036*"truth"')
(5, '0.054*"massacre" + 0.028*"texas" + 0.028*"able" + 0.028*"dollar" + 0.028*"euro"')
(6, '0.054*"ansa" + 0.028*"live" + 0.028*"question" + 0.028*"political" + 0.028*"michela"')
(7, '0.005*"screen" + 0.005*"rapid" + 0.005*"flame" + 0.005*"fire" + 0.005*"extinguished"')
(8, '0.051*"assault_rifle" + 0.027*"opportunity" + 0.027*"propensity" + 0.027*"investimenti" + 0.027*"limit"')
(9, '0.044*"palestinian" + 0.044*"israeli" + 0.044*"united" + 0.044*"nation" + 0.044*"conflict"')


In [49]:
parole_valore=[]
concetti_LDA={}
for idx,lista_t in topics:
  parole_valore=[]
  tmp_p=re.sub('[\"]',"",lista_t)
  tmp_p=tmp_p.split(' + ') 
  coppia=list(t.split('*') for t in tmp_p)
  concetti_LDA['concetto_'+str(idx)]=[]
  for valore,parola in coppia:
    concetti_LDA['concetto_'+str(idx)].append((parola,valore))


In [50]:
concetti_LDA

{'concetto_0': [('fire', '0.050'),
  ('courtroom', '0.026'),
  ('didn', '0.026'),
  ('traveler', '0.026'),
  ('bianchi', '0.026')],
 'concetto_1': [('new', '0.038'),
  ('add', '0.038'),
  ('york', '0.038'),
  ('criminal', '0.038'),
  ('attorney', '0.038')],
 'concetto_2': [('money', '0.033'),
  ('eu', '0.022'),
  ('atm_give', '0.022'),
  ('dynamic', '0.012'),
  ('equation', '0.012')],
 'concetto_3': [('kiev', '0.027'),
  ('ukraine', '0.027'),
  ('war', '0.027'),
  ('not_stop', '0.027'),
  ('mayor', '0.027')],
 'concetto_4': [('not_happy', '0.070'),
  ('not_good', '0.036'),
  ('trump', '0.036'),
  ('riot', '0.036'),
  ('truth', '0.036')],
 'concetto_5': [('massacre', '0.054'),
  ('texas', '0.028'),
  ('able', '0.028'),
  ('dollar', '0.028'),
  ('euro', '0.028')],
 'concetto_6': [('ansa', '0.054'),
  ('live', '0.028'),
  ('question', '0.028'),
  ('political', '0.028'),
  ('michela', '0.028')],
 'concetto_7': [('screen', '0.005'),
  ('rapid', '0.005'),
  ('flame', '0.005'),
  ('fire', '0.

In [ ]:
concetti_LDA['concetto_1']

[('screen', '0.005'),
 ('rapid', '0.005'),
 ('flame', '0.005'),
 ('fire', '0.005'),
 ('extinguished', '0.005')]

In [30]:
def punteggioLDA(concetti_LDA,parole):
  punteggi=dict()
  for key in concetti_LDA.keys():
    punteggio=0
    for i in parole:
      for concetto_parola,valore in concetti_LDA[key]:
        if(i == concetto_parola):
         punteggio+=float(valore)
    punteggi[key]=punteggio
  return punteggi

In [31]:
punteggioLDA(concetti_LDA,parole)

{'concetto_0': 0.05,
 'concetto_1': 0,
 'concetto_2': 0,
 'concetto_3': 0.054,
 'concetto_4': 0,
 'concetto_5': 0,
 'concetto_6': 0,
 'concetto_7': 0.005,
 'concetto_8': 0,
 'concetto_9': 0.044}

In [32]:
def result_TopicLDA(corpus,concetti_LDA):
  risultato = []
  p=punteggioLDA(concetti_LDA,parole)
  massimo = max(p.values())
  print("Risultato di ", corpus)
  for concetto,valore in p.items():
    if valore == massimo:
      risultato.append((concetto, concetti_LDA[concetto]))
  return (risultato)

In [33]:
result_TopicLDA(corpus[14],concetti_LDA)

Risultato di  Term of the day: Computational fluid dynamics Definition: The numerical solution of flow equations in practical problems such as aircraft design or hydraulic structures.


[('concetto_3',
  [('kiev', '0.027'),
   ('ukraine', '0.027'),
   ('war', '0.027'),
   ('not_stop', '0.027'),
   ('mayor', '0.027')])]

In [ ]:
pip install pyLDAvis

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle
from gensim.corpora import Dictionary
pyLDAvis.enable_notebook()

#dictionary = Dictionary.load('dictionary.gensim')
#corpus = pickle.load(open('corpus.pkl', 'rb'))

#lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
lda_display = gensimvis.prepare(ldamodel, corpus2, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


HDP

In [ ]:
hdpmodel = HdpModel(corpus=corpus2, id2word=dictionary)

In [ ]:
hdpmodel.show_topics()

In [ ]:
hdptopics = hdpmodel.show_topics(formatted=False)

In [ ]:
hdptopics[:1]

[(0,
  [('rapid', 0.026388423739676745),
   ('positive', 0.02256674618708974),
   ('public', 0.02015745470582611),
   ('probe', 0.02005634209831726),
   ('hand', 0.0177586347389796),
   ('opportunity', 0.017699737866041),
   ('account', 0.01662331024054387),
   ('beach', 0.016175630379051554),
   ('equation', 0.015340894646176379),
   ('deulofeu', 0.015121851070173248),
   ('cell', 0.014399329012534868),
   ('atm_give', 0.014372568335939927),
   ('monkey', 0.013488524333942353),
   ('able', 0.013469600213790187),
   ('stuck', 0.013410868062506742),
   ('definition', 0.013348230706980912),
   ('nation', 0.013286717207109848),
   ('united', 0.013223687568071373),
   ('question', 0.01284812601737425),
   ('force', 0.011604739482081548)])]

In [ ]:
def punteggioHDP(concetti_HDP,parole):
  punteggi=dict()
  risultato = []
  for key,lista in concetti_HDP:
    punteggio=0
    for i in parole:
      for concetto_parola,valore in lista:
        if(i == concetto_parola):
         punteggio+=valore
    if punteggio!=0:
        risultato.append((key,lista))
    punteggi[key]=punteggio
  return punteggi

In [ ]:
def result_TopicHDP(concetti_HDP,parole):
  risultato = []
  p=punteggioHDP(concetti_HDP,parole)
  massimo = max(p.values())
  #print("Risultato di ", corpus)
  for concetto,valore in p.items():
    if valore == massimo:
      risultato.append((concetto, concetti_HDP[concetto]))
  return (risultato)

In [ ]:
punteggioHDP(hdptopics,parole)

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0.02143479555038231,
 5: 0.014914906246592573,
 6: 0,
 7: 0.02268286324993809,
 8: 0,
 9: 0,
 10: 0.026977405353737945,
 11: 0,
 12: 0,
 13: 0.012773643789369888,
 14: 0.01614796660543255,
 15: 0,
 16: 0.013467613952374564,
 17: 0,
 18: 0.021211031349542803,
 19: 0}

In [ ]:
result_TopicHDP(hdptopics,parole)

[(10,
  (10,
   [('accusation', 0.023182417915928313),
    ('didn', 0.015552451897551627),
    ('napoli', 0.015488321782834387),
    ('region', 0.014906694483395965),
    ('school', 0.014790589307035756),
    ('united', 0.014098983074299833),
    ('elementary', 0.013741346977794014),
    ('fire', 0.013686568664676754),
    ('ukraine', 0.01329083668906119),
    ('not_stop', 0.01288337661873256),
    ('colleague', 0.012828222459810211),
    ('mouse', 0.012779963842487859),
    ('dynamic', 0.012561569984835564),
    ('blocking', 0.01251071145957598),
    ('carefully', 0.01228873421590331),
    ('shown', 0.01219955619880648),
    ('cancer', 0.011913965093047535),
    ('appearing', 0.011138520431367547),
    ('market', 0.010956963753746449),
    ('limit', 0.010838727751770652)]))]